<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#US-Counties" data-toc-modified-id="US-Counties-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>US Counties</a></span></li></ul></div>

# US Counties

The dataset records in this project are indexed by county. Each county is designated a 5-digit FIPS code (https://en.wikipedia.org/wiki/FIPS_county_code). In this exercise, we walk through generating a visualization of the US counties using data provided by the US Census Bureau (https://www.census.gov/geo/maps-data/data/cbf/cbf_counties.html). It exists in the form a shapefile and is already included in this repository.

First we setup our environment:

In [1]:
import os, sys, re, json, time
import matplotlib
from copy import deepcopy
from subprocess import check_output
import netCDF4 as NC
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ipyleaflet import Map, GeoJSON, Choropleth
from ipywidgets import Layout
from astropy.time import Time
from IPython.display import display, HTML, Markdown
from shapely.geometry import Point, Polygon, MultiPolygon, LineString, mapping
import geojson
import seaborn as sns

sns.set_style('white')

%matplotlib inline

Next we read in the US county shapefile into a GeoPandas dataframe:

In [2]:
def get_us_multipolygon():
    """Return multipolygon of US counties."""
    
    counties_shp = os.path.join("geo_data", "counties", "cb_2017_us_county_500k.shp")
    us = gpd.read_file(counties_shp)
    us = us.to_crs(epsg=4326) # reproject to -180, 180
    return us
    
# get geopandas data frame of US county polygons
us_polys = get_us_multipolygon()

Now that we have this dataframe, let's display it in table format:

In [3]:
display(us_polys.head())

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,005,00161528,0500000US01005,01005,Barbour,06,2292144656,50538698,"POLYGON ((-85.74803199999999 31.619181, -85.74..."
1,01,023,00161537,0500000US01023,01023,Choctaw,06,2365869837,19144469,"POLYGON ((-88.47322699999999 31.893856, -88.46..."
2,01,035,00161543,0500000US01035,01035,Conecuh,06,2201948618,6643480,"POLYGON ((-87.42720399999999 31.26436, -87.425..."
3,01,051,00161551,0500000US01051,01051,Elmore,06,1601762124,99965171,"POLYGON ((-86.41333499999999 32.750591, -86.37..."
4,01,065,00161558,0500000US01065,01065,Hale,06,1667907107,32423356,"POLYGON ((-87.870464 32.76244200000001, -87.86..."


Note the field names above. In particular, the `GEOID` field corresponds to the 5-digit FIPS code we discussed earlier. Let's update the dataframe to rename the `GEOID` column to `fips`, set `fips` to be the index of the dataframe. We then display the updated table:

In [4]:
us_fips = us_polys.rename(index=str, columns={"GEOID": "fips"}).set_index('fips')
display(us_fips.head())

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,NAME,LSAD,ALAND,AWATER,geometry
fips,,,,,,,,,
01005,01,005,00161528,0500000US01005,Barbour,06,2292144656,50538698,"POLYGON ((-85.74803199999999 31.619181, -85.74..."
01023,01,023,00161537,0500000US01023,Choctaw,06,2365869837,19144469,"POLYGON ((-88.47322699999999 31.893856, -88.46..."
01035,01,035,00161543,0500000US01035,Conecuh,06,2201948618,6643480,"POLYGON ((-87.42720399999999 31.26436, -87.425..."
01051,01,051,00161551,0500000US01051,Elmore,06,1601762124,99965171,"POLYGON ((-86.41333499999999 32.750591, -86.37..."
01065,01,065,00161558,0500000US01065,Hale,06,1667907107,32423356,"POLYGON ((-87.870464 32.76244200000001, -87.86..."


Next let's display the data on a leaflet map. First we draw a base leaflet map centered on the CONUS (continental US):

In [5]:
# zoom in on the southeastern US
m = Map(center=[31, -85], zoom=6, layout=Layout(height='600px', align_self='stretch'))
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Now we plot the counties onto the leaflet map above (this may take a minute or two):

In [6]:
l = GeoJSON(data=us_fips.__geo_interface__,
            hover_style={'fillColor': 'red'})
m.add_layer(l)